In [1]:
import torch
import torch.nn as nn
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [5]:
import numpy as np

np.linspace(0.0,0.05,17)

array([0.      , 0.003125, 0.00625 , 0.009375, 0.0125  , 0.015625,
       0.01875 , 0.021875, 0.025   , 0.028125, 0.03125 , 0.034375,
       0.0375  , 0.040625, 0.04375 , 0.046875, 0.05    ])